# View backend details

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.43.1
```
</details>

This page explains how to find information about your available backends.

## List or filter backends

<span id="available"></span>
### List backends available to you

You can use either Qiskit or IBM Quantum Platform to view a list of backends available to you, or to search for a particular backend.

<Tabs>
    <TabItem value="Qiskit-available" label="View backends with Qiskit">
        Use the [`QiskitRuntimeService.backends()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#backends) method, as shown in the next code block. This method returns a list of [`IBMBackend`](/docs/api/qiskit-ibm-runtime/ibm-backend) instances.

        To run the following code, be sure you have already authenticated to the service. See [Set up your IBM Cloud account](/docs/guides/cloud-setup) for more details.

        <CodeCellPlaceholder tag="id-initialize" />

        To search for a specific backend, use the [`QiskitRuntimeService.backend()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#backend) method (note that this is singular: *backend*), which takes the name of the backend as the input parameter and returns an [`IBMBackend`](/docs/api/qiskit-ibm-runtime/ibm-backend) instance representing that particular backend:

        <CodeCellPlaceholder tag="id-search" />
    </TabItem>
    <TabItem value="IQP-available" label="View backends on IBM Quantum Platform">
        To view the backends you have access to, navigate to the list of backends on the [Compute resources page](https://quantum.cloud.ibm.com/computers) (note that your selected region might impact the QPUs listed). Click the filter icon and choose either "All my instances" or a specific instance name to see the available QPUs.
    </TabItem>
</Tabs>

In [1]:
# Initialize your account
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()

service.backends()

[<IBMBackend('ibm_fez')>,
 <IBMBackend('ibm_torino')>,
 <IBMBackend('ibm_marrakesh')>]

In [2]:
service.backend("ibm_fez")

<IBMBackend('ibm_fez')>

In [ ]:
# Optionally pass in an instance, region, or both, to
# further filter the backends.
service = QiskitRuntimeService()

service.backends(simulator=False, operational=True, min_num_qubits=100)

[<IBMBackend('ibm_fez')>,
 <IBMBackend('ibm_torino')>,
 <IBMBackend('ibm_marrakesh')>]

<Admonition type="note">
If you are logged in to a specific instance or region, or if you initialized the service with a specific instance or region by using `QiskitRuntimeService()`, only the backends available to you on that instance or region are returned.
</Admonition>

### Filter backends


<Tabs>
    <TabItem value="Qiskit-filter" label="Filter by properties with Qiskit">
        You can filter the available backends by their [configuration](/docs/api/qiskit-ibm-runtime/models-backend-configuration) or [status](/docs/api/qiskit-ibm-runtime/models-backend-status). For more general filters, set the `filters` argument to a function that accepts a backend object and returns `True` if it meets your criteria. Refer to the [API documentation](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#backends) for more details.

        The following code returns only backends that fit these criteria and are available to you _on your currently selected instance_:

        *   Are real quantum devices (`simulator=False`)
        *   Are currently operational (`operational=True`)
        *   Have at least a hundred qubits (`min_num_qubits=100`)

        <CodeCellPlaceholder tag="id-filter" />

        A similar method is [`QiskitRuntimeService.least_busy()`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#least_busy), which takes the same filters as `backends()` but returns the backend that matches the filters and has the least number of jobs pending in the queue:

        <CodeCellPlaceholder tag="id-least-jobs" />

    </TabItem>
    <TabItem value="IQP-filter" label="Filter by properties on IBM Quantum Platform">
        To view the backends you have access to, view the table on the [Compute resources page](https://quantum.cloud.ibm.com/computers). Click the filter icon to see filter options. You can filter by processor type and status. Note also that you can sort the table by any column by hovering over a column title, and then clicking the arrows that appear.
    </TabItem>
</Tabs>

In [ ]:
service.least_busy(operational=True, min_num_qubits=100)

<IBMBackend('ibm_fez')>

## Static backend information

Some information about a backend does not change regularly, such as its name, version, the number of qubits it has, its processor type, and the types of features it supports. This information is available as attributes of the `backend` object. For a full list of attributes, see the [`IBMBackend` API documentation](/docs/api/qiskit-ibm-runtime/ibm-backend).

<Tabs>
  <TabItem value="Qiskit-backend" label="View backend information with Qiskit">

    <CodeCellPlaceholder tag="id-version" />

  </TabItem>
  <TabItem value="IQP-backend" label="View backend information on IBM Quantum Platform">

    Navigate to the [Compute resources](https://quantum.cloud.ibm.com/computers) page to view a table of all QPUs, then click the name of any QPU to open its detailed information card. Find backend information under the Details section.

  </TabItem>
</Tabs>

In [ ]:
backend = service.backend("ibm_fez")

print(
    f"Name: {backend.name}\n"
    f"Version: {backend.backend_version}\n"
    f"No. of qubits: {backend.num_qubits}\n"
    f"Processor type: {backend.processor_type}\n"
)

Name: ibm_fez
Version: 2
No. of qubits: 156
Processor type: {'family': 'Heron', 'revision': '2'}



### QPU versioning

Each QPU has a version number in the form X.Y.Z (major.minor.revision). A circuit compiled for a given version number is guaranteed to run on that QPU. If the revision number changes, the circuit will continue to run. If the major or minor number changes, the circuit is not guaranteed to run, although it may do so.

The revision version number will increment for fixes that do not break the existing compiled circuit.

The conditions under which a version or revision number can change are listed in the following table.

| Major version | Minor version | Revision version |
| --- | --- | --- |
|Sample changes| Warmup / cool-down cycles | QPU software updates |
|Major changes to the control electronics| Swapping out some electronics, if the replacement appreciably affects operation| Manual calibrations to improve fidelities |
|Moving the QPU to a new location, if significant behavior changes result| Dropping a gate for some duration of time due to calibration issues, and corrections cannot readily be done in software| Small electronics changes that don’t affect operation |
| |Changing the direction of a CNOT gate | |

<span id="native-gates-ops"></span>
### Supported instructions

Each [processor family](/docs/guides/processor-types) natively supports a limited set of instructions. This set includes single- and two-qubit gates, as well as non-unitary operations such as measure and reset. Every gate in the circuit must be translated (by the transpiler) to the elements of a QPU's supported instruction set before it can run on the QPU.

You can view the supported instruction sets for a QPU with Qiskit. The IBM Quantum Platform Compute resources page lists only the supported unitary gates (basis gates) for a QPU.


  <Tabs>
  <TabItem value="Qiskit-native" label="Find supported instructions with Qiskit">

    <CodeCellPlaceholder tag="id-gates" />

    </TabItem>
    <TabItem value="IQP-basis" label="Find supported instructions on IBM Quantum Platform">

    Navigate to the [Compute resources](https://quantum.cloud.ibm.com/computers) page to view a table of all QPUs, then click the name of any QPU to open its detailed information card. The supported instructions for that QPU are listed in the Details section.
    </TabItem>
</Tabs>

<span id="native-gates"></span>
### Table of supported instructions

| Operation category |  Name |
| :--- | :--- |
| Single-qubit gates | [`RZ`](/docs/api/qiskit/qiskit.circuit.library.RZGate), [`SX`](/docs/api/qiskit/qiskit.circuit.library.SXGate), [`X`](/docs/api/qiskit/qiskit.circuit.library.XGate), [`ID`](/docs/api/qiskit/qiskit.circuit.library.IGate), [`delay`](/docs/api/qiskit/circuit#qiskit.circuit.Delay) |
| Two-qubit gates | [`CZ`](/docs/api/qiskit/qiskit.circuit.library.CZGate), [`ECR`](/docs/api/qiskit/qiskit.circuit.library.ECRGate)|
| Fractional gates | [`RX`](/docs/api/qiskit/qiskit.circuit.library.RXGate) (single-qubit), [`RZZ`](/docs/api/qiskit/qiskit.circuit.library.RZZGate) (two-qubit) |
| Non-unitary instructions | [`measure`](/docs/api/qiskit/circuit#qiskit.circuit.Measure), [`reset`](/docs/api/qiskit/circuit#qiskit.circuit.Reset) |
| Control flow | [`if_else`](/docs/api/qiskit/qiskit.circuit.IfElseOp) (classical feedforward) |

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend("ibm_kingston")

print(f"Backend: {backend.name}")
print(f"    Processor type: {backend.processor_type}")
print(f"    Supported instructions: {backend.supported_instructions}")

Backend: ibm_kingston
    Processor type: {'family': 'Heron', 'revision': '2'}
    Supported instructions: ['cz', 'id', 'delay', 'measure', 'measure_2', 'reset', 'rz', 'sx', 'x', 'if_else']


## Dynamic backend information

Backends can also have properties that change whenever the backed is calibrated, such as qubit frequency and operation error rates. Backend properties update after the calibration sequence completes. These properties can be used when optimizing quantum circuits or to construct noise models for a classical simulator. See the [Calibration jobs](/docs/guides/calibration-jobs) guide for more information.


<details>
<summary>Click to view qubit properties</summary>

### Qubit properties

The `backend.properties().qubit_property()` returns information about the qubits' physical attributes. It contains a dictionary of various properties of the qubit, each paired with its value and the timestamp of the last calibration.


#### `T1` (Relaxation time)

The $T_1$ time represents the average duration a qubit remains in its excited state $|1\rangle$ before decaying to its ground state $|0\rangle$ due to energy relaxation. This parameter is used to characterize the qubit's energy relaxation behavior, and is expressed in units of seconds (s).


|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().t1(<qubit>)` |
| **View on IBM Quantum Platform** |  Calibration data section, Qubit dropdown menu |

#### `T2` (Dephasing time)

The $T_2$ time denotes the timescale over which a qubit maintains phase coherence of a superposition between the $|0\rangle$ and $|1\rangle$ states. It accounts for both energy relaxation and pure dephasing processes, providing insight into the qubit's coherence properties.


|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().t2(<qubit>)` |
| **View on IBM Quantum Platform** | Calibration data section, Qubit dropdown menu |

#### `frequency`

This parameter specifies the resonant frequency of the qubit, indicating the energy difference between the $|0\rangle$ and $|1\rangle$ states, expressed in hertz (Hz).


|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().frequency(<qubit>)` |
| **View on IBM Quantum Platform** | Calibration data section, graph view  |

#### `anharmonicity`

Anharmonicity is the difference in energy between the first and second excited states of the qubit, also expressed in hertz (Hz).

|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().qubit_property(<qubit>, 'anharmonicity')`|
| **View on IBM Quantum Platform** | Calibration data section, graph view |

#### `readout_error`

The readout error quantifies the average probability of incorrectly measuring a qubit's state. It is commonly calculated as the mean of `prob_meas0_prep1` and `prob_meas1_prep0`, providing a single metric for measurement fidelity.

|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().readout_error(<qubit>)`|
| **View on IBM Quantum Platform** | Details section |

#### `prob_meas0_prep1`

This parameter indicates the probability of measuring a qubit in the 0 state when it was intended to be prepared in the $|1\rangle$ state, denoted as $P(0 | 1)$. It reflects errors in state preparation and measurement (SPAM), particularly measurement errors in superconducting qubits.


|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().qubit_property(<qubit>, 'prob_meas0_prep1')`|
| **View on IBM Quantum Platform** | Calibration data section, table view |

#### `prob_meas1_prep0`

Similarly, this parameter represents the probability of measuring a qubit in the 1 state when it was intended to be prepared in the $|0\rangle$ state, denoted as $P(1 | 0)$. Like prob_meas0_prep1, it reflects SPAM errors, with measurement errors being the predominant contributor in superconducting qubits.

|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().qubit_property(<qubit>, 'prob_meas0_prep0')`|
| **View on IBM Quantum Platform** | Calibration data section, table view |

#### `readout_length`

The `readout_length` specifies the duration of the readout operation for a qubit. It measures the time from the initiation of the measurement pulse to the completion of signal digitization, after which the system is ready for the next operation. Understanding this parameter is crucial for optimizing circuit execution, especially when incorporating mid-circuit measurements.

|  |  |
| --- | --- |
| **View with Qiskit** | `backend.properties().readout_length(<qubit>)`|
| **View on IBM Quantum Platform** | Calibration data section, table view |

The following code examples demonstrate how to list all properties, or retrieve specific properties, of a particular qubit.

    <CodeCellPlaceholder tag="id-properties" />
    <CodeCellPlaceholder tag="id-properties1" />
    <CodeCellPlaceholder tag="id-properties2" />


</details>

In [ ]:
# fundamental physical properties of qubit 1

backend = service.backend("ibm_fez")

backend.qubit_properties(1)

QubitProperties(t1=0.00020833187791710663, t2=0.00023395229316289175, frequency=None)

In [7]:
# calibration data with detailed properties of qubit 0
backend.properties().qubit_property(0)

{'T1': (5.345445238259663e-05,
  datetime.datetime(2025, 11, 25, 6, 52, 23, tzinfo=tzlocal())),
 'T2': (4.569656984603511e-05,
  datetime.datetime(2025, 11, 25, 6, 53, 9, tzinfo=tzlocal())),
 'readout_error': (0.01416015625,
  datetime.datetime(2025, 11, 25, 11, 42, 11, tzinfo=tzlocal())),
 'prob_meas0_prep1': (0.02734375,
  datetime.datetime(2025, 11, 25, 11, 42, 11, tzinfo=tzlocal())),
 'prob_meas1_prep0': (0.0009765625,
  datetime.datetime(2025, 11, 25, 11, 42, 11, tzinfo=tzlocal())),
 'readout_length': (1.56e-06,
  datetime.datetime(2025, 11, 25, 11, 42, 11, tzinfo=tzlocal()))}

In [ ]:
# Retrieve qubit properties
qubit_index = 126  # Replace with your qubit index
qubit_props = backend.properties().qubit_property(qubit_index)

# Access specific properties
t1 = qubit_props.get("T1", (None,))[0]
t2 = qubit_props.get("T2", (None,))[0]
frequency = qubit_props.get("frequency", (None,))[0]
anharmonicity = qubit_props.get("anharmonicity", (None,))[0]
readout_error = qubit_props.get("readout_error", (None,))[0]
prob_meas0_prep1 = qubit_props.get("prob_meas0_prep1", (None,))[0]
prob_meas1_prep0 = qubit_props.get("prob_meas1_prep0", (None,))[0]
readout_length = qubit_props.get("readout_length", (None,))[0]

print(f"Qubit {qubit_index} Properties:")
print(f"  T1: {t1} seconds")
print(f"  T2: {t2} seconds")
print(f"  Frequency: {frequency} Hz")
print(f"  Anharmonicity: {anharmonicity} Hz")
print(f"  Readout Error: {readout_error}")
print(f"  P(0 | 1): {prob_meas0_prep1}")
print(f"  P(1 | 0): {prob_meas1_prep0}")
print(f"  Readout Length: {readout_length} seconds")

Qubit 126 Properties:
  T1: 0.00011004519582612597 seconds
  T2: 6.519695327562977e-05 seconds
  Frequency: None Hz
  Anharmonicity: None Hz
  Readout Error: 0.003662109375
  P(0 | 1): 0.0048828125
  P(1 | 0): 0.00244140625
  Readout Length: 1.56e-06 seconds


<details>
<summary>Click to view instruction properties</summary>

### Instruction properties

2Q error (best) - I
2Q error (layered) - I
Median 2Q error I
Median SX error I
Median readout error I
Median T1 (relaxation time) I
Median T2 (dephasing time) I
Prob meas0 prep1 I
Prob meas1 prep0 I
Readout length (ns) -  I
ID error / √x (sx) error / Pauli-X error / RX error I
Single-qubit gate length (ns) I
Z-axis rotation (RZ) error I
Gate length (ns) I
2Q error I
RZZ error I


The [`backend.target`] attribute is a [`qiskit.transpiler.Target`](/docs/api/qiskit/qiskit.transpiler.Target) object: an object that contains all the information needed to transpile a circuit for that backend. This includes instruction errors and durations. For example, the following cell gets the properties for a [`cz` gate](/docs/api/qiskit/qiskit.circuit.library.CZGate) acting between qubits 1 and 0.

    <CodeCellPlaceholder tag="id-instructions" />

The following cell shows the properties for a measurement operation (including the readout error) on qubit 0.

    <CodeCellPlaceholder tag="id-instructions1" />

</details>

In [9]:
backend.target["cz"][(1, 0)]

InstructionProperties(duration=6.8e-08, error=0.007854516178238763)

In [10]:
backend.target["measure"][(0,)]

InstructionProperties(duration=1.56e-06, error=0.01416015625)

<details>
<summary>Click to view additional properties</summary>

### Additional properties


CLOPS (or CLOPS_h) - A
Status - A
Total pending jobs - A
Topology diagram or coupling map A

</details>

## Find QPU details on IBM Quantum Platform

In addition to finding details programmatically, you can also use IBM Quantum Platform to view information about specific QPUs. Navigate to the [Compute resources](https://quantum.cloud.ibm.com/computers) page to view a table of all QPUs, then click the name of any QPU to open its detailed information card. The card contains details such as error rates, calibration data, and more.

### Details section

The first section of the QPU information card provides the following QPU details:

[Name](#name) | [Qubits](#qubits) | [2Q error (best)](#2q-error-best) | [2Q error (layered)](#2q-error-layered) | [CLOPS (or CLOPS_h)](#clops-or-clops_h) | [Status](#status) | [Region](#region) | [QPU version](#qpu-version) | [Processor type](#processor-type) | [Basis gates](#native-gates-on-platform) | [Total pending jobs](#total-pending-jobs) | [Median 2Q error](#median-2q-error-heron-cz-eagle-ecr) |
[Median SX error](#median-sx-error) | [Median readout error](#median-readout-error) | [Median T1 (relaxation time)](#median-t1-relaxation-time) | [Median T2 (dephasing time)](#median-t2-dephasing-time)

<details>
<summary>Click to view details</summary>
#### Name

The unique name assigned to a specific QPU. QPUs hosted on IBM Cloud&reg; have names that begin with `ibm_*`. All QPUs are given a city name, e.g., `ibm_kingston`. This name does not indicate where the actual QPU is hosted. They are named after IBM&reg; locations around the world.

#### Qubits

The number of physical qubits in a QPU.

#### 2Q error (best)

The lowest two-qubit (2Q) error on any edge of the device from the same batch of measurements used to calculate the median (see [Median 2Q error](#median-2q-error-heron-cz-eagle-ecr)).

#### 2Q error (layered)

Average error per layered gate (EPLG) in a chain of 100 qubits. Average EPLG measures the average gate error in a layered chain of $N$ qubits ($N$=100 here). It is derived from a similar quantity known as the layer fidelity (LF) where EPLG$_{100}$ = 4/5(1-LF$^{\frac{1}{99}}$) and layer fidelity is the process fidelity of the layered chain of $N$ qubits. For details, see the paper [Benchmarking quantum processor performance at scale](https://arxiv.org/abs/2311.05933). Note that in the paper EPLG is defined for process error, but for consistency with the individually reported gate errors here it is quoted for average gate error, thus the factor of 4/5. Find an [example notebook](https://github.com/qiskit-community/qiskit-device-benchmarking/blob/main/notebooks/layer_fidelity_single_chain.ipynb) in the Qiskit Community GitHub.

#### CLOPS (or CLOPS_h)

Circuit layer operations per second, is a measure of how many layers of a 100x100 circuit (hardware-aware circuit) a QPU (quantum processing unit) can execute per unit of time. Find the [CLOPS code](https://github.com/qiskit-community/qiskit-device-benchmarking/tree/main/qiskit_device_benchmarking/clops#readme) in the Qiskit Community GitHub.

#### Status

The QPU status; for example, `Online`, `Paused`, `Offline`, and so on.

#### Region

Location of the data center where your data and experiments will be hosted and processed.

#### QPU version

The version number of a QPU in the form `major.minor.revision`. See [QPU versioning](#qpu-versioning) for details on how this number is assigned.

#### Processor type

Reflects the [topology](#coupling-map) and indicates the approximate qubit count.

<span id="native-gates-on-platform"></span>
#### Basis gates

Each [processor family](/docs/guides/processor-types) has a native gate set. By default, the QPUs in each family only support running the gates and operations in the native gate set. Thus, every gate in the circuit must be translated (by the transpiler) to the elements of this set. Note that the non-unitary operations are not listed here; use the [method in Qiskit](/docs/guides/qpu-information#native-gates-ops) to see all native gates and operations for a QPU. See a list of all native gates in [this table](#native-gates).

#### Total pending jobs

The total number of jobs that you have submitted to this QPU.

#### Median 2Q error (Heron: CZ, Eagle: ECR)

[Average gate fidelity](/docs/api/qiskit/quantum_info#average_gate_fidelity) of the two-qubit operation from randomized benchmarking. Measured in "isolation": batches with a minimum separation of two qubits between edges. This randomized benchmarking uses alternating layers of single-qubit Cliffords and two-qubit gates, and thus the final 2Q error value includes the error of the layer of single-qubit Cliffords. Find an [example notebook](https://github.com/qiskit-community/qiskit-device-benchmarking/blob/main/notebooks/device_rb_w_lf.ipynb) in the Qiskit Community GitHub. Find per-edge data in the [calibration data](#calibration-data) section of the QPU information card.

#### Median SX error

[Average gate fidelity](/docs/api/qiskit/quantum_info#average_gate_fidelity) of the √X (SX) gate from randomized benchmarking, measured simultaneously on all qubits. The randomized benchmarking sequence includes SX, ID, and X gates, and it is assumed their errors are the same.

#### Median readout error

Fidelity of the readout operation. Readout error is measured by preparing the qubit in the 0 (1) state and measuring the probability of an output in the 1 (0) state. The reported value is the average of these two errors. The median is taken over all qubits.

#### Median T1 (relaxation time)

The $T_1$ time represents the average duration a qubit remains in its excited state $|1\rangle$ before decaying to its ground state $|0\rangle$ due to energy relaxation. This parameter is used to characterize the qubit's energy relaxation behavior, and is expressed in units of seconds (s).

#### Median T2 (dephasing time)

The $T_2$ time denotes the timescale over which a qubit maintains phase coherence of a superposition between the $|0\rangle$ and $|1\rangle$ states. It accounts for both energy relaxation and pure dephasing processes, providing insight into the qubit's coherence properties. $T_2$ is reported from a Hahn echo sequence.
</details>

<span id="calibration-data"></span>
### Calibration data section

<Admonition type="note" title="What does `error = 1` mean?">
If the benchmarking of a qubit or edge does not succeed over the course of several days, whether due to poor data quality or other internal factors, the reported error value is considered stale and will be reported as 1. This is not an indication that the qubit or edge is necessarily non-working or that the error is 1; rather, the error is considered undefined and you should proceed with caution when operating that qubit or gate.
</Admonition>

The second section, Calibration data, provides qubit, connectivity, and gate data. You can choose to visualize the information as a map, graph, or table.

You can customize the data that is shown in each view, using the drop-down menus. For example, in the map view, you can choose the data you want to see for qubits and connections. The colored bars associated with the diagram or graph indicate the range that is shown, with the average value marked. The color maximum and minimum change depending on the QPU.

To download calibration data as a CSV file, click the download icon in the upper right corner of the Calibration data section.

The Calibration data section includes the following:

[Topology diagram or coupling map](#coupling-map) | [Readout assignment error](#readout-assignment-error) | [Prob meas0 prep1](#prob-meas0-prep1) | [Prob meas1 prep0](#prob-meas1-prep0) | [Readout length (ns)](#readout-length-ns) | [ID error / √x (sx) error / Pauli-X error / RX error](#finite-duration-discrete) | [Single-qubit gate length (ns)](#single-qubit-gate-length-ns)| [Z-axis rotation (RZ) error](#z-axis-rotation-rz-error) | [Operational](#operational) | [Gate length (ns)](#gate-length-ns) | [2Q error](#2q-error) | [RZZ error](#rzz-error-heron)

<details>
<summary>Click to view calibration details</summary>

<span id="coupling-map"></span>
#### Topology diagram or coupling map

A diagram that indicates the pairs of qubits that support two-qubit gate operations between them. This is also called the coupling map or connectivity. Qubits are represented as circles and the supported two-qubit gate operations are displayed as lines connecting the qubits.

#### Readout assignment error

The readout error quantifies the average probability of incorrectly measuring a qubit's state. It is commonly calculated as the mean of prob_meas0_prep1 and prob_meas1_prep0, providing a single metric for measurement fidelity.

#### Prob meas0 prep1

This parameter indicates the probability of measuring a qubit in the $|0\rangle$ state when it was intended to be prepared in the $|1\rangle$ state, denoted as $P(0|1)$. It reflects errors in state preparation and measurement (SPAM), particularly measurement errors in superconducting qubits.

#### Prob meas1 prep0

Similarly, this parameter represents the probability of measuring a qubit in the $|1\rangle$ state when it was intended to be prepared in the $|0\rangle$ state, denoted as $P(1|0)$. Like prob_meas0_prep1, it reflects SPAM errors, with measurement errors being the predominant contributor in superconducting qubits.

#### Readout length (ns)

The readout_length specifies the duration of the readout operation for a qubit. It measures the time from the initiation of the measurement pulse to the completion of signal digitization, after which the system is ready for the next operation. Understanding this parameter is crucial for optimizing circuit execution, especially when incorporating mid-circuit measurements.

<span id="finite-duration-discrete"></span>
#### ID error / √x (sx) error / Pauli-X error / RX error

Error in the finite-duration discrete one-qubit gates, measured from randomized benchmarking. The randomized benchmarking sequence includes SX, ID, and X gates, and it is assumed their errors are the same. The ID gate is a delay of duration equal to the duration of the √X and X gates. The RX gate is also the same duration as the √X and X gates with variable amplitude, and so it is reported as having the same error as these gates.

#### Single-qubit gate length (ns)

Duration of a single-qubit gate operation.

#### Z-axis rotation (RZ) error

Error in the virtual RZ gate. Reported as all 0 since these are performed in software.

#### Operational

Indicates whether the qubit can be utilized in circuits.

#### Gate length (ns)

Duration of the two-qubit gate operation.

<span id="2q-error"></span>
#### 2Q error (Heron: CZ, Eagle: ECR)

The 2Q error per edge from the same batch of measurements used to calculate the [2Q median](#median-2q-error-heron-cz-eagle-ecr) and [2Q best](#2q-error-best) errors.

#### RZZ error (Heron)

Error in the RZZ gate averaged over the RZZ angles using a variant of randomized benchmarking for arbitrary unitaries.
</details>

### Two-qubit gate error (layered) section

The third section provides the expanded view of the lowest `two-qubit gate error (layered)` measured as a function of the number of qubits in the chain. The final value, at chain length 100, is the value presented in the Details section. In practice, six 100-qubit chains (pre-selected based on expected optimal performance) are measured, and the value reported for number of qubits `N` is the lowest error found in a subset length `N` chain searching over the six 100-qubit chains.

## Next steps

<Admonition type="tip" title="Recommendations">
    - Try the [Grover's algorithm](/docs/tutorials/grovers-algorithm) tutorial.
    - Review the [QiskitRuntime backend API](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#backend) reference.
</Admonition>